# Market Impact Analysis

This notebook demonstrates the Almgren-Chriss impact model and short-term price prediction.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.impact_models.parametric import AlmgrenChrissModel, ImpactParams
from src.impact_models.prediction import PricePredictor
from src.data.synthetic import generate_synthetic_lob
from src.features.microstructure import MicrostructureFeatures

%matplotlib inline

## 1. Almgren-Chriss Impact Curves

In [ ]:
params = ImpactParams(eta=0.5, gamma=0.05)
model = AlmgrenChrissModel(params)

rates = np.linspace(0, 1000, 100)
temp_impacts = [model.calculate_temporary_impact(r, 0.02) for r in rates]

plt.figure(figsize=(10, 5))
plt.plot(rates, temp_impacts)
plt.title('Temporary Impact vs Trading Rate')
plt.xlabel('Trading Rate (shares/sec)')
plt.ylabel('Cost per Share')
plt.grid(True)
plt.show()

## 2. Price Prediction Model

In [ ]:
df = generate_synthetic_lob(n_events=2000, seed=123)
ofi = MicrostructureFeatures.calculate_ofi(df, time_window='10s')
tfi = MicrostructureFeatures.calculate_tfi(df, time_window='10s')

# Merge features
features = ofi.join(tfi).fillna(0)
# Add dummy price for target generation (resampled)
features['price'] = df.set_index(pd.to_datetime(df['timestamp'], unit='s'))['price'].resample('10s').last().ffill()

predictor = PricePredictor()
predictor.train(features)

print("Model Trained:", predictor.is_trained)
print("Coefficients:", predictor.model.coef_)